In [ ]:
#|default_exp eval_overlap

# Evaluate Overlaps

> Routines to check overlaps between nightlight scans and aoi

In [ ]:
#| eval: false
#| hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import requests
import json
from fastcore.all import L
import pandas as pd
from typing import Any
from urllib.parse import urlparse
from pathlib import Path
import numpy as np
import re
from shapely.geometry import Polygon, box
from shapely import wkt
import geopandas as gpd
import sqlite3 as sql3
from pandas.io import sql 
import sqlalchemy as sqalc
import geowrangler.area_zonal_stats as azs

In [ ]:
#| exporti
from ntlights_damage_assessment.fetch_coords import search_dates_aoi 

In [ ]:
#| hide
import matplotlib.pyplot as plt

In [ ]:
#| eval: false
aoi = gpd.read_file('../data/admin_areas/ph_admin_bbox_wgs84.geojson')
aoi_planar = aoi.to_crs('EPSG:3857')
aoi_planar['bound_area'] = aoi_planar.geometry.area

In [ ]:
%%time
#| eval: false
phl_matched20180501 = search_dates_aoi(aoi, date_param='20180501')

CPU times: user 866 ms, sys: 613 ms, total: 1.48 s
Wall time: 10.2 s


In [ ]:
#| eval: false
phl_matched_planar = phl_matched20180501.to_crs('EPSG:3857')
phl_matched_planar['section_area'] = phl_matched_planar.geometry.area

In [ ]:
%%time
#| eval: false
results = azs.create_area_zonal_stats(phl_matched_planar, aoi_planar, aggregations=[])

CPU times: user 48.9 ms, sys: 0 ns, total: 48.9 ms
Wall time: 49.2 ms


In [ ]:
#| eval: false
results.head()

,href,item_href,stem,suffix,base_url,folder,product_id,start_date,first_scantime,end_scantime,...,create_datetime,data_origin,data_domain,vflag_file,vflag_href,cache_stem,asset_href,geometry,section_area,intersect_area_sum
80,./SVDNB_npp_d20180501_t1634142_e1639546_b33728...,https://globalnightlight.s3.amazonaws.com/npp_...,SVDNB_npp_d20180501_t1634142_e1639546_b33728_c...,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,1634142,1639546,...,20180501223954897862,noac,ops,npp_d20180501_t1634142_e1639546_b33728.vflag.c...,https://globalnightlight.s3.amazonaws.com/npp_...,SVDNB_npp_d20180501_t1634142_e1639546_b33728_c...,https://globalnightlight.s3.amazonaws.com/npp_...,"POLYGON ((13246796.765 2155183.158, 13246796.7...",1.437277e+13,2.127466e+11
81,./SVDNB_npp_d20180501_t1639558_e1645362_b33728...,https://globalnightlight.s3.amazonaws.com/npp_...,SVDNB_npp_d20180501_t1639558_e1645362_b33728_c...,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,1639558,1645362,...,20180501224536066966,noac,ops,npp_d20180501_t1639558_e1645362_b33728.vflag.c...,https://globalnightlight.s3.amazonaws.com/npp_...,SVDNB_npp_d20180501_t1639558_e1645362_b33728_c...,https://globalnightlight.s3.amazonaws.com/npp_...,"POLYGON ((12912838.293 229.689, 12912838.293 2...",1.088441e+13,2.237799e+12
82,./SVDNB_npp_d20180501_t1645375_e1651161_b33728...,https://globalnightlight.s3.amazonaws.com/npp_...,SVDNB_npp_d20180501_t1645375_e1651161_b33728_c...,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,1645375,1651161,...,20180501225118045996,noac,ops,npp_d20180501_t1645375_e1651161_b33728.vflag.c...,https://globalnightlight.s3.amazonaws.com/npp_...,SVDNB_npp_d20180501_t1645375_e1651161_b33728_c...,https://globalnightlight.s3.amazonaws.com/npp_...,"POLYGON ((12356240.839 -2272782.155, 12356240....",1.039730e+13,6.358422e+10
89,./SVDNB_npp_d20180501_t1816398_e1822184_b33729...,https://globalnightlight.s3.amazonaws.com/npp_...,SVDNB_npp_d20180501_t1816398_e1822184_b33729_c...,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,1816398,1822184,...,20180502002220226537,noac,ops,npp_d20180501_t1816398_e1822184_b33729.vflag.c...,https://globalnightlight.s3.amazonaws.com/npp_...,SVDNB_npp_d20180501_t1816398_e1822184_b33729_c...,https://globalnightlight.s3.amazonaws.com/npp_...,"POLYGON ((10463798.364 1804965.959, 10463798.3...",1.321192e+13,8.253388e+11
90,./SVDNB_npp_d20180501_t1822196_e1828000_b33729...,https://globalnightlight.s3.amazonaws.com/npp_...,SVDNB_npp_d20180501_t1822196_e1828000_b33729_c...,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,1822196,1828000,...,20180502002801307723,noac,ops,npp_d20180501_t1822196_e1828000_b33729.vflag.c...,https://globalnightlight.s3.amazonaws.com/npp_...,SVDNB_npp_d20180501_t1822196_e1828000_b33729_c...,https://globalnightlight.s3.amazonaws.com/npp_...,"POLYGON ((10018520.400 -333881.539, 10018520.4...",1.045368e+13,1.839721e+12


In [ ]:
#| eval: false
aoi_planar

,geometry,bound_area
0,"POLYGON ((14093600.302 511201.726, 14093600.30...",2.600698e+12


In [ ]:
#| eval: false
tot_aoi_area = aoi_planar.bound_area.sum(); tot_aoi_area
results['pct_covered'] = results['intersect_area_sum'] / tot_aoi_area 
sorted_results = results.sort_values(by='pct_covered',ascending=False)
sorted_results

In [ ]:
#| export


def compute_overlaps(aoi, matched_results):
    aoi_planar = aoi.to_crs('EPSG:3857')
    aoi_planar['bound_area'] = aoi_planar.geometry.area
    matched_planar = matched_results.to_crs('EPSG:3857')
    matched_planar['section_area'] = matched_planar.geometry.area
    results = azs.create_area_zonal_stats(matched_planar, aoi_planar, aggregations=[])
    tot_aoi_area = aoi_planar.bound_area.sum()
    results['pct_covered'] = results['intersect_area_sum'] / tot_aoi_area 
    sorted_results = results.sort_values(by='pct_covered',ascending=False)
    return sorted_results

In [ ]:
%%time
#| eval: false
sorted_results = compute_overlaps(aoi,phl_matched20180501) 

CPU times: user 37.2 ms, sys: 6.45 ms, total: 43.6 ms
Wall time: 41.8 ms


In [ ]:
#| eval: false
sorted_results[['pct_covered']].head()

,pct_covered
81,0.860461
90,0.707395
89,0.317353
80,0.081804
82,0.024449


In [ ]:
#| eval: false
aoi_ph = gpd.read_file('../data/admin_areas/phl_admbnda_adm3_psa_namria_20200529.shp')
aoi_ph_planar = aoi_ph.to_crs('EPSG:3857')
ax = plt.axes()
ax = sorted_results[:2].plot(ax=ax, facecolor='none', edgecolor='blue')
ax = aoi_ph_planar.plot(ax=ax,color='red')